In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import stan
import h5py
import nest_asyncio
nest_asyncio.apply()

In [ ]:
class AR1:
    """
    LinReg model with Stan
    """

    def __init__(self, data: np.ndarray, N_tilde: int):
        self.model_name = "StanLinReg"
        self.data = data
        self.N = len(self.data)
        self.N_tilde = N_tilde

    def fit(self):
        stan_code = """
            data {
                int<lower=0> N;
                vector[N] y;
                int<lower=0> N_tilde;
            }
            parameters {
                real alpha;
                real beta;
                real<lower=0> sigma;
            }
            model {
                alpha ~ normal(0, 10);
                beta ~ normal(0, 10);
                sigma ~ cauchy(0, 2.5);
                y[2:N] ~ normal(alpha + beta * y[1:(N-1)], sigma);
            }
            generated quantities {
                vector[N_tilde] y_tilde;
                real i = y[N];
                for (n in 1:N_tilde) {
                    y_tilde[n] = normal_rng(alpha + beta * i, sigma);
                    i = y_tilde[n];
                    }
            }
        """

        data = {
            "N": self.N,
            "y": self.data,
            "N_tilde": self.N_tilde,
        }
        model = stan.build(program_code=stan_code, data=data)
        self.fit = model.sample(num_chains=1, num_samples=5 * 10**1)

    def predict(self):
        predictions = self.fit["y_tilde"]  # this is coming from the model object
        predictions = self.format_sample(predictions, self.date, self.location)
        return predictions
        
    def format_quantiles(self, data: pd.DataFrame):
        """
        Formats the samples form the stan model (precessed througth the format_sample function) into a percentiles.

        Parameters
        -----------
        data: pd.DataFrame
            This is the dataframe that is returned from the format_sample function.
        
        Returns
        --------
        data: pd.DataFrame
        """
        def createQuantiles(x):
            quantiles = np.array(
                [
                    0.010,
                    0.025,
                    0.050,
                    0.100,
                    0.150,
                    0.200,
                    0.250,
                    0.300,
                    0.350,
                    0.400,
                    0.450,
                    0.500,
                    0.550,
                    0.600,
                    0.650,
                    0.700,
                    0.750,
                    0.800,
                    0.850,
                    0.900,
                    0.950,
                    0.975,
                    0.990,
                ]
            )
            quantileValues = np.percentile(x["value"], q=100 * quantiles)
            return pd.DataFrame(
                {"quantile": list(quantiles), "value": list(quantileValues)}
            )

        dataQuantiles = (
            data.groupby(["forecast_date", "target_end_date", "location"])
            .apply(lambda x: createQuantiles(x))
            .reset_index()
            .drop(columns="level_3")
        )

        return dataQuantiles

In [ ]:
with h5py.File('./Site_Callahan_Divide_Wind_Energy_Center_wind_actuals_2018.h5', 'r') as f:
    actuals = pd.DataFrame(f['actuals'][...])
    actual_time_steps = pd.to_datetime(f['time_index'][...].astype(str), errors='coerce', format='%Y-%m-%d %H:%M:%S')

In [ ]:
model = AR1(actuals[0].to_numpy(), 24)
model.fit()

In [ ]:
def format_quantiles(data: pd.DataFrame, length: int):
    """
    Formats the samples form the stan model (precessed througth the format_sample function) into a percentiles.

    Parameters
    -----------
    data: pd.DataFrame
        This is the dataframe that is returned from the format_sample function.
    
    Returns
    --------
    data: pd.DataFrame
    """
    def createQuantiles(x):
        quantiles = np.array(
            [
                0.010,
                0.025,
                0.050,
                0.100,
                0.150,
                0.200,
                0.250,
                0.300,
                0.350,
                0.400,
                0.450,
                0.500,
                0.550,
                0.600,
                0.650,
                0.700,
                0.750,
                0.800,
                0.850,
                0.900,
                0.950,
                0.975,
                0.990,
            ]
        )
        quantileValues = np.percentile(x["value"], q=100 * quantiles)
        return pd.DataFrame(
            {"quantile": list(quantiles), "value": list(quantileValues)}
        )

    data_predictions = {
        "prediction_idx": [],
        "sample": [],
        "value": [],
    }

    for N_tilde, samples in enumerate(data):
        for n, sample in enumerate(samples):
            data_predictions["prediction_idx"].append(length + N_tilde)
            data_predictions["sample"].append(n)
            data_predictions["value"].append(sample)
    
    data_predictions = pd.DataFrame(data_predictions)

    dataQuantiles = (
        data_predictions.groupby(["prediction_idx"])
        .apply(lambda x: createQuantiles(x))
        .reset_index()
        .drop(columns="level_1")
    )

    return dataQuantiles

In [ ]:
preds = format_quantiles(model.predict(), 100000)

In [ ]:
def plot_single_predictions(data: pd.DataFrame, preds: pd.DataFrame):
    """
    Plot the predictions and the 95% confidence interval for a single prediction.
    
    Parameters
    -----------
    data: pd.DataFrame
        The original dataframe of data from `get_data` function.
    preds: pd.DataFrame
        This is a dataframe that is in the same format as a dataframe from `format_quantiles`
    to_plot: str
        The name of the paremeter you want to plot: cases, deaths, hosps.
    
    Returns
    --------
    A plt plot
    """

    low = preds.loc[preds["quantile"] == 0.025, "value"]
    mid = preds.loc[preds["quantile"] == 0.500, "value"]
    high = preds.loc[preds["quantile"] == 0.975, "value"]
    x = preds.loc[preds["quantile"] == 0.025, "prediction_idx"]

    # plt.figure(figsize=(10, 6), dpi=150)
    plt.plot(data, label="Truth Data")
    plt.plot(x, mid, label="Predictions", color="black")
    plt.fill_between(
        x, low, high, color="red", alpha=0.5, label="95% Confidence Interval"
    )
    plt.legend()
    plt.xlabel("idx")
    plt.ylabel("")
    plt.show()

In [ ]:
plot_single_predictions(actuals[0][99000:100000], preds)